In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install pandas openpyxl

In [ ]:
import pandas as pd
import numpy as np

# Create a dictionary with messy data
data = {
    'Customer_ID': ['1001', '1002', '1003', '1004', '1002', '1005', '1006'],
    'Name': ['Alice Smith', 'Bob Jones', 'Charlie', 'David Brown', 'Bob Jones', np.nan, 'Frank White'],
    'Join_Date': ['15-01-2023', '02/20/2023', '2023.03.10', '05-04-2023', '02/20/2023', '01/01/2023', 'Not available'],
    'Phone': ['(555) 123-4567', '555.987.6543', '555 111 2222', 'Unknown', '555.987.6543', '555-000-0000', '555-123-4567'],
    'Email': ['alice@example.com', 'bob@example.com', np.nan, 'david@example', 'bob@example.com', 'eve@example.com', 'frank@example.com']
}

# Create DataFrame
df = pd.DataFrame(data)

# Save to CSV
df.to_csv('/content/drive/MyDrive/Colab Notebooks/DEProjects/customer_dump.csv', index=False)
print("Dirty data file 'customer_dump.csv' created successfully!")

Dirty data file 'customer_dump.csv' created successfully!


In [16]:
import pandas as pd
from datetime import datetime
import re

# --- STEP 1: EXTRACT ---
print("--- Loading Data ---")
try:
  df  = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DEProjects/customer_dump.csv')
  print("Data Loaded Successfully.")
  print(f"Original shape: {df.shape}")
except FileNotFoundError:
  print("File not found. Please check the file path.")
  exit()
except Exception as e:
  print(f"An error occurred: {e}")

# --- STEP 2: TRANSFORM ---
print("\n--- Cleaning Data ---")

# 1. Remove Duplicates
# We check for duplicates based on Customer_ID and Name to be safe
initial_rows = len(df)
df.drop_duplicates(inplace=True)
print(f"Removed {initial_rows - len(df)} duplicate rows.")

# 2. Handle Null Values
# Fill missing names with "Unknown" and missing emails with "No Email"
df['Name'] = df['Name'].fillna('Unknown')
df['Email'] = df['Email'].fillna('No Email')
print("Null values handled.")

# 3. Standardize Dates
# We define a helper function to handle the messy formats
def clean_date(date_str):
  try:
    # pd.to_datetime is powerful. "errors='coerce'" turns unparseable data (like 'Not available') into NaT (Not a Time)
    # format='mixed' allows pandas to guess different formats in the same column
    return pd.to_datetime(date_str, format='mixed',dayfirst=True)
  except:
    return None

# Apply the cleaning function
df['Join_Date'] = df['Join_Date'].apply(clean_date)
print("Dates standardized.")

# Drop rows where the date could not be parsed (optional, depending on business logic)
# Here, we will fill invalid dates with today's date or a placeholder, but let's just drop nulls for this exercise
df = df.dropna(subset=['Join_Date'])

# Convert to ISO format string (YYYY-MM-DD) for the final report
df['Join_Date'] = df['Join_Date'].dt.strftime('%Y-%m-%d')

# 4. Clean Phone Numbers
# Goal: Remove anything that isn't a digit (parentheses, dots, dashes, spaces)
def clean_phone(phone_str):
  if pd.isna(phone_str) or phone_str == 'Unknown':
    return "Unknown"
  # Regex: \D matches any non-digit character. We replace them with an empty string.
  return re.sub(r'\D', '',str(phone_str))
df['Phone'] = df['Phone'].apply(clean_phone)

# --- STEP 3: LOAD ---
print("\n--- Saving Data ---")

# Generate filename with today's date
today_str = datetime.now().strftime('%Y-%m-%d')
output_file = f"/content/drive/MyDrive/Colab Notebooks/DEProjects/cleaned_data_{today_str}.xlsx"

print(f"Saving cleaned data to {output_file}")
print("\n--- Final Data Preview ---")
print(df.head(10))
df.to_excel(output_file, index=False)
print("Data saved successfully!")

--- Loading Data ---
Data Loaded Successfully.
Original shape: (7, 5)

--- Cleaning Data ---
Removed 1 duplicate rows.
Null values handled.
Dates standardized.

--- Saving Data ---
Saving cleaned data to /content/drive/MyDrive/Colab Notebooks/DEProjects/cleaned_data_2025-12-25.xlsx

--- Final Data Preview ---
   Customer_ID         Name   Join_Date       Phone              Email
0         1001  Alice Smith  2023-01-15  5551234567  alice@example.com
1         1002    Bob Jones  2023-02-20  5559876543    bob@example.com
2         1003      Charlie  2023-03-10  5551112222           No Email
3         1004  David Brown  2023-04-05     Unknown      david@example
5         1005      Unknown  2023-01-01  5550000000    eve@example.com
Data saved successfully!
